In [1]:
import pandas as pd
import re
import nltk
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import ast
import spacy

nltk.download('stopwords')
nltk.download('wordnet')
stemmer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
df = pd.read_csv('data/korea_herald_2015.csv')
target = df[:][['title', 'description', 'body', 'keyword', 'cluster']]
for i in range(target.shape[0]):
    target.title[i] = preprocess_text(target.title[i])
    target.body[i] = preprocess_text(target.body[i])
    target.description[i] = preprocess_text(target.description[i])


FileNotFoundError: [Errno 2] No such file or directory: 'data/koreaherald_2015_bow_ver.csv'

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
n_cluster = 15
cluster_vector = []
cluster_feature = []
for i in range(n_cluster):
    cluster_idx = target.cluster == i
    vectorizer = CountVectorizer(max_features=1500, ngram_range=(, 6), min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
    X_count = vectorizer.fit_transform(target[cluster_idx].body).toarray()
    cluster_vector.append(X_count)
    cluster_feature.append(vectorizer.get_feature_names())

In [17]:
from sklearn.feature_extraction.text import TfidfTransformer
cluster_vector_tfidf = [TfidfTransformer().fit_transform(x).toarray() for x in cluster_vector]
df_tfidf = [pd.DataFrame(cluster_vector_tfidf[i], columns= cluster_feature[i])for i in range(n_cluster)]

In [21]:
df.keyword[3]

'higher medical costs ,much higher medical costs ,higher medical costs if ,thursday the welfare ministry ,emergency patients will face'

In [109]:
def preprocessing(rawtext):
    document = rawtext
    document = re.sub(r'said', '', document)

    dates = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday', 'week', 'next', 'month', 'year']
    for date in dates:
        document = re.sub(r'{}'.format(date), '', document)
    
    document = re.sub(r'\s+', ' ', document)
    return document
    
def remove_duplicates(candidates):
    result = []
    for cand in candidates:
        is_duplicate = False
        for cand2 in candidates:
            if cand == cand2:
                continue
            if cand in cand2:
                is_duplicate = True
                break
        if not is_duplicate:
            result.append(cand)
    return result
    

In [111]:
updated_cluster = [[0], [1, 2, 13], [3], [4], [5], [6], [7, 8, 11], [9], [10], [12], [14]]

for i in updated_cluster:
    keyword_set = target[target['cluster'].isin(i)].keyword
    keyword_set = [preprocessing(keyword) for keyword_list in keyword_set for keyword in keyword_list]
    ketword_text = ''.join(keyword_set)

    vectorizer = CountVectorizer(max_features=1500, ngram_range=(3, 5), min_df=0.7, max_df=5, stop_words=stopwords.words('english'))
    X_count = vectorizer.fit_transform([preprocessing(ketword_text)]).toarray()

    result = pd.DataFrame(X_count, columns= vectorizer.get_feature_names())
    result = list(result.sum(axis=0).sort_values(ascending=False).keys())[:100]

    print('=========={}=========='.format(i))
    print(remove_duplicates(result)[:50])
print('======================')



==========[0]==========
['president park geun', 'south korea korea', 'south korean president', 'day south korea', 'north korea nuclear', 'reform south korea', 'pentagon press secretary', 'south korea south korea', 'south korea last', 'reform president park', 'capsized fishing boat', 'korean president park', 'fourth largest economy', 'third largest steelmaker', '2015 world science', 'daejeon 2015 world', 'annual landing drill', 'four new patients', 'guard helicopter crashed', 'anti pyongyang leaflets', 'north korea north korea', 'korea top financial regulator', 'president south korea', 'fatalities occurring since', 'asia fourth largest', 'corruption president park', 'new attack submarine', 'joint military drills', 'fishing boat capsized', 'talks south korea', 'south korean firms', 'president park reform', 'college entrance exam', 'aid south korea', 'south korea top', 'change conference president', 'south korea second largest', 'test free semester', 'fifth largest shareholder', 'south ko

In [18]:
n_cluster = 15

def remove_duplicates(candidates):
    result = []
    for cand in candidates:
        is_duplicate = False
        for cand2 in candidates:
            if cand == cand2:
                continue
            if cand in cand2:
                is_duplicate = True
                break
        if not is_duplicate:
            result.append(cand)
    return result

for i in range(n_cluster):
    # remove word 'cluster'
    tf_idf_ne = df_tfidf[i].sum(axis=0).sort_values(ascending=False).keys()[:100]
    print("==============topic candidate: {}==============".format(i))
    print(remove_duplicates(list(tf_idf_ne))[:10])
    # break
    # print(list(tf_idf))
print("=============================================")

==============topic candidate: 0==============
['president park geun hye', 'said south korea', 'middle east respiratory syndrome', 'south korea said', 'south korea top', 'foreign ministry said', 'south korea japan', 'ruling saenuri party', '1950 53 korean war', 'south korea united states']
==============topic candidate: 1==============
['japanese prime minister shinzo abe', 'president park geun hye', 'japan colonial rule', 'wartime sexual slavery', '50th anniversary normalization', 'japan 1910 45 colonial rule', 'sexual enslavement korean women', 'world heritage committee', 'relations south korea', 'south korean government']
==============topic candidate: 2==============
['south korea japan', 'japanese prime minister shinzo abe', 'south korea china', 'islamic state group', 'collective self defense', 'prime minister abe', '70th anniversary end world war ii', 'comfort women issue', 'cheong wa dae', 'foreign minister yun byung se']
==============topic candidate: 3==============
['seoul me